In [2]:
import torch 
import pandas as pd
import cv2
import numpy as np

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

Using cache found in C:\Users\adamf/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-26 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [22]:
img = cv2.imread('../samples/frame1.jpg')
out = model(img, size=[720, 1280])

In [23]:
df = out.pandas().xyxy[0]
df

,xmin,ymin,xmax,ymax,confidence,class,name
0,1000.250916,525.268372,1071.285645,561.339294,0.792455,2,car
1,518.679199,482.752808,743.213501,613.166504,0.734270,7,truck
2,1114.333252,514.285217,1164.992188,544.084167,0.707595,2,car
3,0.000000,550.890137,107.816895,622.506470,0.676201,2,car
4,1063.990845,519.335144,1116.287231,553.466370,0.642054,2,car
5,1186.751099,482.454010,1256.434692,541.856873,0.333085,7,truck
6,82.560028,564.528870,236.841248,620.699524,0.324158,2,car


In [24]:
def draw_boxes(boxes, labels, image):
    """
    Draws the bounding box around a detected object.
    """
    print(boxes)
    for i, box in enumerate(boxes):
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            (255, 0, 0), 2
        )
        cv2.putText(image, str(labels[i]), (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2,
                    lineType=cv2.LINE_AA)
    return image

image = draw_boxes([x[1][0:4] for x in df.iterrows()], df['name'], img)
cv2.imshow('img', image)
cv2.waitKey(0)
# closing all open windows
cv2.destroyAllWindows()

[xmin    1000.250916
ymin     525.268372
xmax    1071.285645
ymax     561.339294
Name: 0, dtype: object, xmin    518.679199
ymin    482.752808
xmax    743.213501
ymax    613.166504
Name: 1, dtype: object, xmin    1114.333252
ymin     514.285217
xmax    1164.992188
ymax     544.084167
Name: 2, dtype: object, xmin           0.0
ymin    550.890137
xmax    107.816895
ymax     622.50647
Name: 3, dtype: object, xmin    1063.990845
ymin     519.335144
xmax    1116.287231
ymax      553.46637
Name: 4, dtype: object, xmin    1186.751099
ymin      482.45401
xmax    1256.434692
ymax     541.856873
Name: 5, dtype: object, xmin     82.560028
ymin     564.52887
xmax    236.841248
ymax    620.699524
Name: 6, dtype: object]


In [25]:
pd.read_clipboard()

,Read times - Voltage graph,Voltage(V) - Voltage graph,Read times - Current graph,Current(A) - Current graph,Unnamed: 4
0,0,5.216,0,0.4014,NaN
1,1,5.216,1,0.6860,NaN
2,2,5.189,2,0.6860,NaN
3,3,5.189,3,0.6167,NaN
4,4,5.187,4,0.6167,NaN
...,...,...,...,...,...
1847,1847,5.176,1847,1.0967,NaN
1848,1848,5.180,1848,1.0967,NaN
1849,1849,5.180,1849,1.0684,NaN
1850,1850,5.152,1850,1.0684,NaN
